In [ ]:
    """
    Create folder for workers to verify images
    """
    folder_names = ['josh_james_amadu',
                    'jaewon_james_josh',
                    'jaewon_james_amadu',
                    'josh_jaewon_amadu']
    annotator_allocation = [['Jaewon','Jamila','Jonathan','Mia','Faiz','Alex'],
                            ['Amadu','Aidan', 'Sunny'],
                            ['Josh', 'Jackson'],
                            ['James','Qianyu', 'Connor', 'Celine', 'group_unreviewed_unverfied_images']]

    folder_annotator_list = []

    verification_dir = os.path.join(home_directory,'verification') #create verification folder 

    for i in range(len(folder_names)):    
        print(folder_names[i])
        #create verification subfolder for each group
 
        folder_annotator_list.append([folder_names[i],annotator_allocation[i]])

    return(folder_annotator_list, verification_dir)



def seperate_images_for_verification_update_tracking(folder_annotator_list, verification_dir, set_number, tile_img_annotation_annotator):
    """
    Move images to verifcation folder
    """
    for i in range(len(folder_annotator_list)): #iterate over folders
        print("folder",folder_annotator_list[i][0]) #the current folder
        count = 0
        for ii in range(len(folder_annotator_list[i][1])): #iterate over annotator
            #print("annotator",folder_annotator_list[i][1][ii]) #the current annotator
            for iii in np.where(tile_img_annotation_annotator[:, 4] == folder_annotator_list[i][1][ii])[0]:

                if len(tile_img_annotation_annotator[iii,5]) == 0:
                    tile_img_annotation_annotator[iii,5] = folder_annotator_list[i][0].split("_")[0].capitalize()#coverage
                    tile_img_annotation_annotator[iii,6] = folder_annotator_list[i][0].split("_")[1].capitalize()#quality
                    tile_img_annotation_annotator[iii,7] = folder_annotator_list[i][0].split("_")[2].capitalize()#class
                    
                    shutil.copy(tile_img_annotation_annotator[iii, 2], 
                                os.path.join(verification_dir, "verify_" + folder_annotator_list[i][0] + "_" + set_number, "chips")) #copy images

                    shutil.copy(os.path.join(tile_img_annotation_annotator[iii, 2].rsplit("/",1)[0],"chips_positive_xml",
                                             tile_img_annotation_annotator[iii, 3]), 
                                os.path.join(verification_dir, "verify_" + folder_annotator_list[i][0] + "_" + set_number, "chips_xml")) #copy annotations

                    count += 1 #count the files allocated to each 
        print(count)
    return tile_img_annotation_annotator




In [2]:
import argparse

import numpy as np
import pandas as pd
from pathlib import Path
import os
import shutil

from glob import glob
import data_eng.az_proc as ap

In [38]:
"""
Create folder for workers to verify images
"""
folder_names = ['not alvarez',     # Poonacha       Niculescu    Sunny                     
                "not cleave",      # Alvarez        Poonacha     Niculescu                  
                "not sunny",       # Cleave         Alvarez      Poonacha                      
                'not niculescu',   # Sunny          Cleave       Alvarez             
                'not poonacha',    # Niculescu      Sunny        Cleave                           
                 "anyone",         # Tang          Sunny#2
                 "anyone",         # Tang                   
                ]                            
annotator_allocation = [['Alvarez','Amadu','Jonathan','James'],
                        ['Cleave'],
                        ['Sunny'],
                        ['Niculescu','Nayak','Feinberg'],
                        ['Poonacha','Tang'],
                        ['Kang','Mia'],
                        ['Josh','Jaewon',"Shen",'Katpally'],
]

tile_img_annotation_annotator = np.load('C:/Users/rapiduser/cred/AST_dataset/data_download_and_preprocessing/outputs/tile_img_annotation_annotator.npy')
folder_annotator_list = []

verification_dir = os.path.join(":C",'verification') #create verification folder 

for i in range(len(folder_names)):    
    print(folder_names[i])
    #create verification subfolder for each group

    folder_annotator_list.append([folder_names[i],annotator_allocation[i]])

for i in range(len(folder_annotator_list)): #iterate over folders
    print("folder",folder_annotator_list[i][0]) #the current folder
    count = 0
    for ii in range(len(folder_annotator_list[i][1])): #iterate over annotator
        print("annotator",folder_annotator_list[i][1][ii]) #the current annotator
        for iii in np.where(tile_img_annotation_annotator[:, 4] == folder_annotator_list[i][1][ii])[0]:

            if len(tile_img_annotation_annotator[iii,5]) == 0:
                count += 1 #count the files allocated to each 
            #else:
                #print(tile_img_annotation_annotator[iii,5]) 
    print(count)


not alvarez
not cleave
anyone
not niculescu
not poonacha
anyone
not sunny
folder not alvarez
annotator Alvarez
annotator Amadu
annotator Jonathan
annotator James
940
folder not cleave
annotator Cleave
964
folder anyone
annotator Kang
annotator Mia
991
folder not niculescu
annotator Niculescu
annotator Nayak
annotator Feinberg
1189
folder not poonacha
annotator Poonacha
annotator Tang
1267
folder anyone
annotator Josh
annotator Jaewon
annotator Shen
annotator Katpally
1007
folder not sunny
annotator Sunny
1022


In [ ]:
                if len(tile_img_annotation_annotator[iii,5]) == 0:
                    tile_img_annotation_annotator[iii,5] = folder_annotator_list[i][0].split("_")[0].capitalize()#coverage
                    tile_img_annotation_annotator[iii,6] = folder_annotator_list[i][0].split("_")[1].capitalize()#quality
                    tile_img_annotation_annotator[iii,7] = folder_annotator_list[i][0].split("_")[2].capitalize()#class
                    
                    shutil.copy(tile_img_annotation_annotator[iii, 2], 
                                os.path.join(verification_dir, "verify_" + folder_annotator_list[i][0] + "_" + set_number, "chips")) #copy images

                    shutil.copy(os.path.join(tile_img_annotation_annotator[iii, 2].rsplit("/",1)[0],"chips_positive_xml",
                                             tile_img_annotation_annotator[iii, 3]), 
                                os.path.join(verification_dir, "verify_" + folder_annotator_list[i][0] + "_" + set_number, "chips_xml")) #copy annotations

                    count += 1 #count the files allocated to each 
        print(count)
    return tile_img_annotation_annotato